### **RED WINE QUALITY**



#Data Preprocessing

In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics.cluster import contingency_matrix
import numpy as np
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.cluster import KMeans

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
data = pd.read_csv("/content/winequality-red.csv")

In [40]:
data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [41]:
#Standardization with 2 methods (z-scoring, range normalization)

In [42]:
rank= pd.DataFrame(preprocessing.MinMaxScaler(feature_range=(0,100)).fit_transform(data), columns=data.columns)
z_scoring= pd.DataFrame(preprocessing.StandardScaler().fit_transform(data), columns=data.columns)
ranges=(data-np.mean(data))/(np.max(data)-np.min(data))
ranges

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,-0.081384,0.117931,-0.270976,-0.043754,-0.019143,-0.068661,-0.044056,0.077336,0.156604,-0.058772,-0.157382,-0.127205
1,-0.045986,0.241219,-0.270976,0.004191,0.017585,0.128522,0.072552,0.003915,-0.087491,0.013085,-0.095844,-0.127205
2,-0.045986,0.159027,-0.230976,-0.016357,0.007568,-0.012323,0.026616,0.018599,-0.040247,-0.004880,-0.095844,-0.127205
3,0.254899,-0.169740,0.289024,-0.043754,-0.020812,0.015846,0.047817,0.092021,-0.118987,-0.046796,-0.095844,0.072795
4,-0.081384,0.117931,-0.270976,-0.043754,-0.019143,-0.068661,-0.044056,0.077336,0.156604,-0.058772,-0.157382,-0.127205
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,-0.187579,0.049438,-0.190976,-0.036904,0.004229,0.227114,-0.008720,-0.135586,0.109360,-0.046796,0.011849,-0.127205
1595,-0.214127,0.015191,-0.170976,-0.023206,-0.042515,0.325705,0.016015,-0.119433,0.164478,0.060989,0.119541,0.072795
1596,-0.178729,-0.012206,-0.140976,-0.016357,-0.019143,0.184860,-0.022854,-0.073912,0.085738,0.055001,0.088772,0.072795
1597,-0.214127,0.080260,-0.150976,-0.036904,-0.020812,0.227114,-0.008720,-0.093736,0.203848,0.031049,-0.034305,-0.127205


In [43]:
#For the convenience we will create list of 2 versions of standardization

In [44]:
sets=[z_scoring, ranges]
scatters=np.zeros(2)
scatters

array([0., 0.])

Data scatter

In [45]:
for i in range(len(sets)):
  scatters[i]=sum(sum(sets[i].values**2))
scatters

array([19188.       ,   359.0599926])

SVD

In [46]:
us, ss, vs = list(), list(), list()
for i in sets:
  u, s, v = np.linalg.svd(i)
  us.append(u)
  ss.append(s)
  vs.append(v)
ss

[array([70.64522028, 59.8729437 , 51.87473939, 44.07741345, 39.44931595,
        32.54973115, 31.44344379, 28.44099541, 25.64528711, 22.89853947,
        16.97555853,  9.75546767]),
 array([10.74572067,  8.86019604,  6.94988416,  4.96596577,  4.7591985 ,
         4.0931432 ,  4.04643273,  3.44570854,  3.21622184,  2.76313915,
         2.15216087,  1.36512497])]

Load vectors

In [47]:
vs[0][:, 0]

array([ 0.48788336, -0.00417321,  0.16482854,  0.23109808, -0.07877938,
        0.0555313 , -0.30721496,  0.20052866, -0.17457815,  0.18295601,
       -0.25643792,  0.63857976])

In [48]:
vs[0][:, 1]

array([-0.26512898,  0.33896786,  0.22708884, -0.04185824,  0.29937933,
        0.297287  , -0.62623369,  0.14612614, -0.06022334, -0.15510563,
        0.37716123,  0.00466168])

In [49]:
vs[1][:, 0]

array([ 0.49837736, -0.12870815,  0.14462475, -0.1865083 ,  0.28330633,
       -0.33700306,  0.0450038 ,  0.36080835, -0.08471947,  0.05671207,
        0.13536795, -0.56973053])

In [50]:
vs[1][:, 1]

array([-0.24552142, -0.21349029,  0.13285519, -0.20260857,  0.41521605,
       -0.11306672,  0.47636504, -0.12865451,  0.53646013, -0.15648121,
       -0.30736248, -0.00536444])